<a href="https://colab.research.google.com/github/badriadhikari/2019-Spring-DL/blob/master/course_content/module5/Speeding_up_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random , os
import multiprocessing as mp
from queue import Empty
import math
import time

# Table of Contents

>[Table of Contents](#scrollTo=2HdfYln7P5og)

>[Parallelism In Python](#scrollTo=gWQm3RSn52aK)

>>[Should we always parallelize?](#scrollTo=5xONRRht-mX9)

>>[To Sync, or not to Sync](#scrollTo=bFtBJTN-_Kfl)

>[Cython](#scrollTo=WYUD-hXj_SjD)

>>[Normal Fibonacci](#scrollTo=oxYWdXa7_TGt)

>>[Simple Cython](#scrollTo=0mbFx_F7AQ-V)

>>[Using Cython Data Types](#scrollTo=vuiY4EW8_4z_)

>>[Going past the ~80x speedup](#scrollTo=-JKkNKflNNMv)

>[Using Numba to accelerate Python codes](#scrollTo=LNR1Ko74Jv3-)

>>[What's Just in time Compiler?](#scrollTo=LNR1Ko74Jv3-)

>>[When and where to use accelerators to your code?](#scrollTo=LNR1Ko74Jv3-)

>>[Using jit](#scrollTo=LNR1Ko74Jv3-)

>>>[Array sum](#scrollTo=LNR1Ko74Jv3-)

>>>[What's a decorator?](#scrollTo=CKgaNFPhNaPL)

>>>[Comparing with numpy?](#scrollTo=NifOtkOpN4sH)

>>[When does numba compile things?](#scrollTo=Z50nmALYOBa2)

>>[Before going to see how Numba actually works, lets see about vectorize.](#scrollTo=Z50nmALYOBa2)

>>>[Defining ufuncs using vectorize](#scrollTo=Z50nmALYOBa2)

>>>[Passing multiple signatures](#scrollTo=fDOxlFgvSMaW)

>>[Finally, When will Numba fail in accelerating your code?](#scrollTo=Gv_OdHSkSzyQ)

>>[What's all this pyobject business?](#scrollTo=c85QCDMvUdTO)

>>[Figuring out what isn't working](#scrollTo=c85QCDMvUdTO)

>>[Forcing nopython mode](#scrollTo=F1FzuR5mVFcZ)

>>[Other compilation flags](#scrollTo=Oh-lYoUHVnch)



# Parallelism In Python

Notice how easy it is to parallelize a program. Let us see how to do it with an example.

This first snippet will run on a single core. To see the same in action, execute them in your terminal and monitor your core usage using `htop`

In [0]:
def f(x) :
    return x **3

y = range(int(3*10**6))
results = [f(i) for i in y]

This next snippet will be executed as 4 subprocesses which your CPU will automatically divide amongst the threads/cores available in your PC

In [0]:
def f(x) :
    return x**3

p=mp.Pool(4)

y = range(int(3*10**6))

results = p.map(f,y)

And **voila!** You've just parallelized your first piece of code. When used correctly, it can lead to significant improvements in your program execution. Here are some standard benchmarked result on counting the sum of primes from $10^5$ to $5\times10^5$

**Benchmarked results**

* Single Threaded: **41 minutes, 57 seconds**
* Multi Threaded (8 threads): **106 minutes, 29 seconds**
* MultiProcessing (8 Processes): **6 minutes, 22 seconds**

**Woah!** As expected, the multiprocessing version ran significantly faster, but why did the multithreaded version run much slower? And what really is the difference between the multithreading and multiprocessing library? I'll leave that to you to read about

## Should we always parallelize?

Let's time the previous piece of code, shall we

In [0]:
def f(x) :
    return x**3

p=mp.Pool(4)

y = range(int(3*10**6))

print('Time to execute as a single process: ', end='')
%timeit results = [f(i) for i in y]
print('Time to execute as multiple processes: ', end='')
%timeit results = p.map(f,y)

Time to execute as a single process: 1 loop, best of 3: 1.61 s per loop
Time to execute as multiple processes: 1 loop, best of 3: 2.2 s per loop


Well, that didn't work out quite as planned, did it? Adding to that, we have not added what is generally considered the biggest overhead in multiprocessing, the creation of the subprocesses (with `mp.Pool()` in this case)

<font size=5>Choose where to parallelize wisely.</font>

## To Sync, or not to Sync

If the multiprocessing library is spawning independent processes, are they truly running asynchronous? Since they're independent subprocesses, the common sense answer to that should be yes. Let's test that, shall we?

In [0]:
def f() :
    start = time.time ()
    time.sleep(2)
    return time.time() - start

p = mp.Pool(processes =1)

t1 = time.time()
result = p.apply(f)
for i in range(4):
    time.sleep(0.5)
print('Time to execute f:', result)
print('Time for whole code to run: ', time.time()-t1)

Time to execute f: 2.002092123031616
Time for whole code to run:  4.006682634353638


Aah. There's a difference in the timings. Now why is that?

It turns out the answer lies in the fact that most functions in the Multiprocessing library naturally block the interpretter when executing a command. If we want to let the subprocesses continue on independently, there are only a couple of changes to make to the above code

In [0]:
def f() :
    start = time.time ()
    time.sleep(2)
    return time.time() - start

p = mp.Pool(processes =1)

t1 = time.time()
result = p.apply_async(f)
for i in range(4):
    time.sleep(0.5)
print('Time to execute f:', result.get())
print('Time for whole code to run: ', time.time()-t1)

# Some async functions to work with: result.ready(), result.get()(is blocking), apply_async()

Time to execute f: 2.0020923614501953
Time for whole code to run:  2.005807876586914


# Cython

## Normal Fibonacci

In [0]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)
%timeit fib(20)

100 loops, best of 3: 2.95 ms per loop


## Simple Cython

In [0]:
!pip install cython
%load_ext Cython

In [0]:
%%cython
def fib_cython(n):
    if n<2:
        return n
    return fib_cython(n-1)+fib_cython(n-2)

In [0]:
%timeit fib_cython(20)

1000 loops, best of 3: 1.08 ms per loop


## Using Cython Data Types

In [0]:
%%cython
cpdef long fib_cython_type(long n):
    if n<2:
        return n
    return fib_cython_type(n-1)+fib_cython_type(n-2)


In [0]:
%timeit fib_cython_type(20)

10000 loops, best of 3: 30.9 µs per loop


## Going past the ~80x speedup

It might seem impossible to break this barrier as Cython does the speedup by converting the code to C code (in a sense) and so beating a low level language performance might be tough. But it is still possible to blow this performance out of the water by caching the function itself

In [0]:
from functools import lru_cache as cache

@cache(maxsize=None)
def fib_cache(n):
    if n<2:
        return n
    return fib_cache(n-1)+fib_cache(n-2)
  
%timeit fib_cache(20)

The slowest run took 148.39 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 113 ns per loop


Yes, you can ignore the warning, because caching the function was the very point of it. The fundamental problem with this piece of code is that it is the poorly written version of calculating fibonacci numbers, where to compute fib(20), fib(19) and every number below it is called multiple times. Hence, caching the function itself results in really good speed ups (**30,000x speedup in our case**)

# Using Numba to accelerate Python codes

Numba is a JIT (**Just in time**) for python.

## What's Just in time Compiler?

A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently.

## When and where to use accelerators to your code?

Before we dive in to accelerating python codes with numba, we must first find where to accelerate. How to accelerate is covered in this notebook anyways :p. 

First profile your code to find where are the hotspots or bottlenecks which comsumes a lot of time. Essentially, once that part of the code is found, if we optimize that particular sections of code, we can achieve indeed good results instead of naively accelerating the code.

Example profilers are `Cprofiler, SnakeVix (complicated visualisation)`. I would want all of you guys to read about profiling codes first then continue this tutorial.

## Using jit

This will be made clear with an example

### Array sum

The function below is a naive `sum` function that sums all the elements of a given array. Without using any vectorization or numpy functions.

In [0]:
def sum_array(inp):
    J, I = inp.shape
    
    #this is a bad idea
    mysum = 0
    for j in range(J):
        for i in range(I):
            mysum += inp[j, i]
            
    return mysum

In [0]:
!pip install numba

    100% |████████████████████████████████| 1.9MB 703kB/s 
    100% |████████████████████████████████| 14.8MB 90kB/s 


In [0]:
import numpy as np

In [0]:
arr = np.random.random((300,300))
sum_array(arr)

45113.22616731501

In [0]:
plain = %timeit -o sum_array(arr)

100 loops, best of 3: 18.6 ms per loop


This is a plain implementation of summing an array. If anyone knew about numpy and saw this code they would have fainted :p. So for the purpose of illustration, lets make this `jit` function and see the amazing power of `numba`

In [0]:
from numba import jit

In [0]:
# Want the jitted function to be stored in sum_array_numba
# Note sum_array_numba is a jitted function
sum_array_numba = jit()(sum_array)

In [0]:
sum_array_numba(arr)

# Note the result is same as above. So the function's working hasn't changed due to jit compiling

45113.22616731501

In [0]:
jitted = %timeit -o sum_array_numba(arr)

10000 loops, best of 3: 99.3 µs per loop


In [0]:
plain.best/jitted.best

187.52893490611777

OMG!! The speedup we get is almost **200x** for one line of extra code

Here we used `jit` as a function call. But more commonly, we use it as a decorator.

### What's a decorator?

Decorators provide a simple syntax for calling higher-order functions.

Put simply, decorators wrap a function, modifying its behavior.

It does something before and after the function without modifying the function (taken as argument) and saves the function in its original name

The extra paranthesis there was because this jit was a decorator and takes the arguments in extra paranthesis

In [0]:
@jit
def sum_array(inp):
    I, J = inp.shape
    
    mysum = 0
    for i in range(I):
        for j in range(J):
            mysum += inp[i, j]
            
    return mysum

In [0]:
sum_array(arr)

45113.22616731501

In [0]:
%timeit sum_array(arr)

10000 loops, best of 3: 99.7 µs per loop


### Comparing with numpy?

In [0]:
%timeit arr.sum()

The slowest run took 16.56 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 46.5 µs per loop


Generally, NumPy is faster than Numba in this case since Numpy has a lot of optimization behind (calls C and other things)

## When does `numba` compile things?

The first time you call the function.

So just putting the decorator above the function doesnt compile the function. Compilation consumes processor resources and can potentially be a overhead to the program running time.

So the function is compiled when the function is called.

It tries to remove the overhead of initial compilation step.

## Before going to see how Numba actually works, lets see about vectorize.

### Defining `ufuncs` using `vectorize`

**Whats a `ufunc`??**

A universal function (or ufunc for short) is a function that operates on ndarrays in an element-by-element fashion, supporting array broadcasting, type casting, and several other standard features. That is, a ufunc is a “vectorized” wrapper for a function that takes a fixed number of specific inputs and produces a fixed number of specific outputs.

In [0]:
import math

In [0]:
def trig(a,b):
  return math.sin(a**2)*math.exp(b)

In [0]:
trig(1, 1)

2.2873552871788423

This works fine for scalars, but what about vectors??

In [0]:
a = np.ones((5,5))
b = np.ones((5,5))

In [0]:
trig(a, b)

TypeError: ignored

Oh! we get an error. This is because  math library can operate only on scalars and cannot operate on vectors. But vectorization is essential so that we can make better use of CPU's multi-core architecture. So here we are going to see the amazing power of `vectorize` a numba function.

In [0]:
from numba import vectorize
import numpy

In [0]:
# similar to jit, calling as a function call
vec_trig = vectorize()(trig)

In [0]:
vec_trig(a, b)

array([[2.28735529, 2.28735529, 2.28735529, 2.28735529, 2.28735529],
       [2.28735529, 2.28735529, 2.28735529, 2.28735529, 2.28735529],
       [2.28735529, 2.28735529, 2.28735529, 2.28735529, 2.28735529],
       [2.28735529, 2.28735529, 2.28735529, 2.28735529, 2.28735529],
       [2.28735529, 2.28735529, 2.28735529, 2.28735529, 2.28735529]])

And just like that, the scalar function `trig` is now a NumPy `ufunc` called `vec_trig`

Note that this is a "Dynamic UFunc" with no signature given.

Lets compare with numpy

In [0]:
def numpy_trig(a, b):
    return numpy.sin(a**2) * numpy.exp(b)

In [0]:
a = np.random.random((1000, 1000))
b = np.random.random((1000, 1000))

In [0]:
%timeit vec_trig(a, b)

10 loops, best of 3: 41.4 ms per loop


In [0]:
%timeit numpy_trig(a, b)

10 loops, best of 3: 39.8 ms per loop


They are almost same. The small difference can be neglected because CPU might be in different state (process its running) when we ran these two code snippets

What happens if we do specify a signature? Is there a speed boost?

In [0]:
vec_trig = vectorize('float64(float64, float64)')(trig)
# This 'float64(float64, float64)' is basically dtype of retuned value(dtype of i/p in order)

In [0]:
%timeit vec_trig(a, b)

10 loops, best of 3: 40.9 ms per loop


`Parallel processing` and making program run on `cuda enabled GPU` is very easy here.

Using `target` argument in `vectorize` we can specify what we want.

If no target element is given, it runs in a single core on your CPU.

`target = 'parallel'`: Runs the code(function) parallely on all the available cores in your CPU

`target = 'cuda'`: Runs the vectorized function in a cuda enabled GPU. (if you have one)

In [0]:
vec_trig = vectorize('float64(float64, float64)', target='parallel')(trig)
vec_trig_cuda = vectorize('float32(float32, float32)', target='cuda')(trig)

In [0]:
%timeit vec_trig(a, b)

10 loops, best of 3: 21.3 ms per loop


Run this line of code if you have a cuda enabled GPU. 

```python
%timeit vec_trig_cuda(a.astype('float32'), b.astype('float32'))

```

Automatic multicore operations!

**Note**: `target='parallel'` is not always the best option.  There is overhead in setting up the threading, so if the individual scalar operations that make up a `ufunc` are simple you'll probably get better performance in serial.  If the individual operations are more expensive (like trig!) then parallel is (usually) a good option.

**Note**: `target='cuda'` is used to enable Nvidia GPU for this vectorisation operation

### Passing multiple signatures

If you use multiple signatures, they have to be listed in order of most specific -> least specific

In [0]:
@vectorize(['int32(int32, int32)',
            'int64(int64, int64)',
            'float32(float32, float32)',
            'float64(float64, float64)'])
def trig(a, b):
    return math.sin(a**2) * math.exp(b)

In [0]:
trig(1, 1)

2

In [0]:
trig(1., 1.)

2.2873552871788423

In [0]:
trig.ntypes

4

In [0]:
# Even conditional operations are allowed in vectorize functions
def truncate(a, amin, amax):
    if a < amin:
        a = amin
    elif a > amax:
        a = amax
    return a

vec_truncate_serial = vectorize(['float64(float64, float64, float64)'])(truncate)
vec_truncate_par = vectorize(['float64(float64, float64, float64)'], target='parallel')(truncate)


In [0]:
a = numpy.random.random((5000))

In [0]:
amin = .2
amax = .6

In [0]:
%timeit vec_truncate_serial(a, amin, amax)

The slowest run took 10.73 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 18.1 µs per loop


In [0]:
%timeit vec_truncate_par(a, amin, amax)

The slowest run took 14.32 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 75 µs per loop


In [0]:
%timeit numpy.clip(a, amin, amax)

The slowest run took 7.39 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 9.23 µs per loop


Parallelizing a code neednt always be the most optimal solution. One must remember the overhead of setting the processors and transfer of data. Parallelising code helps if we have computationally intensive tasks.

## Finally, When will Numba fail in accelerating your code?

There are essentially 2 modes in which numba operates. When you `jit` a function remember we never gave the data type of variables that this function will encounter. So numba infers the type of arguments and run smoothly. 

Numba was made to `accelerate mathematical and scientific computing`. What if we do things which are natural in python but not so mathematically sound, like adding 2 strings.

**Note: ** 
Numba examines Python bytecode and then translates this into an 'intermediate representation'. To view this IR, run (compile) a jitted function with inspect_types method.

In [0]:
def add_strings(a, b):
    return a + b

In [0]:
add_strings_jit = jit()(add_strings)

In [0]:
add_strings_jit('a', 'b')

'ab'

In [0]:
add_strings_jit.inspect_types()

add_strings (str, str)
--------------------------------------------------------------------------------
# File: <ipython-input-56-d1008a9d4aa2>
# --- LINE 1 --- 
# label 0
#   del b
#   del a
#   del $0.3

def add_strings(a, b):

    # --- LINE 2 --- 
    #   a = arg(0, name=a)  :: pyobject
    #   b = arg(1, name=b)  :: pyobject
    #   $0.3 = a + b  :: pyobject
    #   $0.4 = cast(value=$0.3)  :: pyobject
    #   return $0.4

    return a + b




## What's all this pyobject business?  

This means it has been compiled in `object` mode.  This can be a faster than regular python if it can do loop lifting, but not that fast.  
We want those `pyobjects` to be `int64` or another type that can be inferred by Numba. Your best bet is forcing `nopython` mode: this will throw an error if Numba finds itself in object mode, so that you _know_ that it can't give you speed.

For the full list of supported Python and NumPy features in `nopython` mode, see the Numba documentation here: [Supported datatypes](http://numba.pydata.org/numba-doc/latest/reference/pysupported.html)

## Figuring out what isn't working

In [0]:
@jit
def add(a, b):
    for i in range(100):
        c = i
        f = i + 7
        l = c + f
        
    return a + b

add('a', 'b')

'ab'

Numba actually has a command line tool that will help us in  identifying where numba falls back to pyobject mode from nopython mode.

```bash
!numba --annotate-html fail.html program.py
```

This will display with colors where numba accelerated the code and where it failed and jumped back to pyobject mode. 

## Forcing `nopython` mode

In [0]:
add_strings_jit = jit(nopython=True)(add_strings)

In [0]:
add_strings_jit('a', 'b')

TypingError: ignored

In [0]:
from numba import njit
# njit always jits a function in nopython mode. unlike jit 
# which tries to run the code in nopython mode if it fails
# falls back to pyobject mode
# This version of jit forces jit to execute in nopython mode. Other words accelerate the program, 
# if its not possible for numba to accelerate, it throws an error

In [0]:
add_strings_jit = njit(add_strings)

In [0]:
add_strings_jit('a', 'b')

TypingError: ignored

## Other compilation flags

There are two other main compilation flags for `@jit`

```python
cache=True
```

if you don't want to always want to get dinged by the compilation time for every run. This will actually save the compiled function into something like a `pyc` file in your `__pycache__` directory, so even between sessions you should have nice fast performance.

```python
nogil=True
```

This releases the GIL.  Note, however, that it doesn't do anything else, like make your program threadsafe.  You have to manage all of those things on your own (use `concurrent.futures`).